In [ ]:
!wget http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip
!unzip /content/BX-CSV-Dump.zip

--2022-04-15 05:53:30--  http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip
Resolving www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)... 132.230.105.133
Connecting to www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)|132.230.105.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘BX-CSV-Dump.zip’

BX-CSV-Dump.zip     100%[===================>]  24.88M  16.6MB/s    in 1.5s    

2022-04-15 05:53:32 (16.6 MB/s) - ‘BX-CSV-Dump.zip’ saved [26085508/26085508]

Archive:  /content/BX-CSV-Dump.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
!pip install -q findspark
!pip install recommenders
!pip install surprise

     |████████████████████████████████| 335 kB 4.9 MB/s 
     |████████████████████████████████| 636 kB 41.2 MB/s 
     |████████████████████████████████| 11.8 MB 1.0 MB/s 
     |████████████████████████████████| 1.5 MB 52.3 MB/s 
     |████████████████████████████████| 12.4 MB 34.8 MB/s 
     |████████████████████████████████| 61 kB 6.1 MB/s 
     |████████████████████████████████| 310 kB 47.8 MB/s 
     |████████████████████████████████| 4.0 MB 25.3 MB/s 
     |████████████████████████████████| 197 kB 24.8 MB/s 
     |████████████████████████████████| 749 kB 53.7 MB/s 
     |████████████████████████████████| 10.9 MB 26.7 MB/s 
     |████████████████████████████████| 381 kB 46.7 MB/s 
     |████████████████████████████████| 6.6 MB 21.3 MB/s 
     |████████████████████████████████| 895 kB 48.4 MB/s 
     |████████████████████████████████| 77 kB 3.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705357 sha256=c56e4742a670a40be134c4f7fa272c7ecc8

In [ ]:
import os
import numpy as np
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
#Use as many threads as your spark local machine have
#spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder.master("local[4]")\
          .appName("StructureAPI")\
          .config('spark.driver.memory','15g')\
          .config('spark.submit.deployMode','client')\
          .config('spark.executor.memory','2g')\
          .config('spark.executor.cores',4)\
          .config('spark.yarn.queue','short')\
          .getOrCreate() 

In [ ]:
!ls

BX-Book-Ratings.csv  BX-Users.csv		spark-3.2.1-bin-hadoop3.2.tgz
BX-Books.csv	     sample_data
BX-CSV-Dump.zip      spark-3.2.1-bin-hadoop3.2


# Book Crossing Dataset

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField
from recommenders.datasets.spark_splitters import spark_random_split
from pyspark.ml.feature import OneHotEncoder, StringIndexer

import pandas as pd

## Apply ALS model and compute RSME for the predictions

In [ ]:
#Load Data
Book_rating_pd = pd.read_csv('/content/BX-Book-Ratings.csv', sep=';',  encoding='latin-1')
ratings = spark.createDataFrame(Book_rating_pd)

stringIndexer = StringIndexer(inputCol="ISBN", outputCol="ISBN-index")
model_index = stringIndexer.fit(ratings)
indexed = model_index.transform(ratings)
#spilt trainSet and test Set
(training, test) = indexed.randomSplit([0.8, 0.2])

In [ ]:
#Train model
als = ALS(maxIter = 10, regParam = 0.01, userCol = "User-ID", itemCol = "ISBN-index", ratingCol = "Book-Rating",
          coldStartStrategy = "drop")
model_als = als.fit(training)
#Predict
predictions = model_als.transform(test)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "Book-Rating",
                                predictionCol = "prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
#userRecs = model_als.recommendForAllUsers(10)
#userRecs.show(truncate=False)

# Generate top 10 movie recommendations for a specified set of users
users = indexed.select(als.getUserCol()).distinct().limit(2)
userSubsetRecs = model_als.recommendForUserSubset(users, 10)
userSubsetRecs.show(truncate=False)

Root-mean-square error = 5.663791556456616


/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|User-ID|recommendations                                                                                                                                                                                           |
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|276813 |[{184140, 43.029785}, {114700, 17.437967}, {111235, 13.96839}, {244247, 13.950374}, {286087, 13.950374}, {61164, 13.798506}, {11673, 13.025088}, {22024, 12.97398}, {17430, 12.24416}, {47271, 12.128245}]|
|277548 |[{130159, 13.490131}, {61164, 12.465095}, {114700, 12.1880045}, {220113, 11.906603}, {6046, 11.83733}, {9135, 11.73258}, {16548, 11.727808}

##NDCG Metric

In [ ]:
from recommenders.evaluation.spark_evaluation import SparkRankingEvaluation

In [ ]:
rank_eval = SparkRankingEvaluation(test, predictions, k = 10, col_user="User-ID", col_item="ISBN", 
                                    col_rating="Book-Rating", col_prediction="prediction",
                                    relevancy_method="top_k")

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
print("NDCG:", round(rank_eval.ndcg_at_k(), 12) )

NDCG: 0.940327885116


##Hit Rate Metric

In [ ]:
import pandas as pd
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise import Reader
from surprise import Dataset
from surprise import accuracy

from collections import defaultdict

In [ ]:
Book_rating_pd = pd.read_csv('/content/BX-Book-Ratings.csv', sep=';',  encoding='latin-1')

In [ ]:
Book_rating_pd.rename(columns = {"User-ID" :"userId", "ISBN":"bookId", "Book-Rating":"rating"}, inplace=True)

In [ ]:
Book_rating_pd['bookId'] = Book_rating_pd['bookId'].astype(int)

ValueError: ignored

In [ ]:
print(Book_rating_pd)

         userId       bookId  rating
0        276725   034545104X       0
1        276726   0155061224       5
2        276727   0446520802       0
3        276729   052165615X       3
4        276729   0521795028       6
...         ...          ...     ...
1149775  276704   1563526298       9
1149776  276706   0679447156       0
1149777  276709   0515107662      10
1149778  276721   0590442449      10
1149779  276723  05162443314       8

[1149780 rows x 3 columns]


In [ ]:
#convert spark DF to pandas DF
indexed_HitRate = indexed.toPandas()
indexed_HitRate.rename(columns = {"User-ID:userId", "ISBN-index:bookId", "Book-Rating:rating"}, inplace=True)

# pandas DF to Dataset Surprise
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(indexed_HitRate[['userId', 'bookId', 'rating']], reader)

KeyboardInterrupt: ignored

In [ ]:
HR_ALS_spark = ALS(maxIter=10, regParam=0.01, userCol="userId", itemCol="bookId", ratingCol="rating", coldStartStrategy="drop")
LOOCV = LeaveOneOut(n_splits=1, random_state=1)

for trainSet, testSet in LOOCV.split(data):
  #Train model
  newTrainSet = spark.createDataFrame(trainSet.build_testset()).selectExpr("_1 as userId", "_2 as bookId", "_3 as rating")
  model = HR_ALS_spark.fit(newTrainSet)
  #Predict
  newTestSet = spark.createDataFrame(testSet).selectExpr("_1 as userId", "_2 as bookId", "_3 as rating")
  leftOutPredictions = model.transform(newTestSet).collect()
  #Build prediction for all rating
  bigTestSet = trainSet.build_anti_testset(0)
  newBigTestSet = spark.createDataFrame(bigTestSet).selectExpr("_1 as userId", "_2 as bookId", "_3 as rating")
  allPredictions = model.transform(newBigTestSet)

In [ ]:
list_allPredictions = allPredictions.collect()

###GET TOP N

In [ ]:
def GetTopN(predictions, n=10, minimumRating=4.0):
    topN = defaultdict(list)

    for userID, movieID, actualRating, estimatedRating in predictions:
        if (estimatedRating >= minimumRating):
            topN[int(userID)].append((int(movieID), estimatedRating))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userID)] = ratings[:n]

    return topN

In [ ]:
#compute top 10 recom
topNPredicted = GetTopN(list_allPredictions, n=10)

### Calculate Hit Rate

In [ ]:
def hit_rate(topNPredicted, leftOutPred):
    hits = 0
    total = 0

 # For each left-out rating
    for leftOut in leftOutPred:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        # Is it in the predicted top 10 for this user?
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            if (int(leftOutMovieID) == int(movieID)):
                hit = True
                break
        if (hit) :
            hits += 1

        total += 1

    # Compute overall precision
    return hits/total

In [ ]:
print("Hit rate: ", round(hit_rate(topNPredicted, leftOutPredictions), 12))